<a href="https://colab.research.google.com/github/GabrielTrentino/Competicoes/blob/master/Pre%C3%A7o_imoveis_S%C3%A3o_Paulo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importando as bibliotecas neceessárias:

In [2]:
# importar os pacotes necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor
# importar o dataset para um dataframe
df = pd.read_csv('sao-paulo-properties-april-2019.csv')

# ver as 5 primeiras entradas
df.head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.542818,-46.485665
3,1000,200,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.547171,-46.483014
4,1300,410,55,2,2,1,1,1,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.525025,-46.482436


#Análise Exploratória Inicial:

In [3]:
df.isnull().sum()

Price               0
Condo               0
Size                0
Rooms               0
Toilets             0
Suites              0
Parking             0
Elevator            0
Furnished           0
Swimming Pool       0
New                 0
District            0
Negotiation Type    0
Property Type       0
Latitude            0
Longitude           0
dtype: int64

In [4]:
df.dtypes

Price                 int64
Condo                 int64
Size                  int64
Rooms                 int64
Toilets               int64
Suites                int64
Parking               int64
Elevator              int64
Furnished             int64
Swimming Pool         int64
New                   int64
District             object
Negotiation Type     object
Property Type        object
Latitude            float64
Longitude           float64
dtype: object

In [5]:
df.describe().round(2)

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,Latitude,Longitude
count,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00
mean,287737.78,689.88,84.37,2.31,2.07,0.98,1.39,0.35,0.15,0.51,0.02,-22.08,-43.60
std,590821.42,757.65,58.44,0.78,0.96,0.83,0.83,0.48,0.35,0.50,0.12,5.87,11.49
min,480.00,0.00,30.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,-46.75,-58.36
25%,1858.75,290.00,50.00,2.00,2.00,1.00,1.00,0.00,0.00,0.00,0.00,-23.59,-46.68
50%,8100.00,500.00,65.00,2.00,2.00,1.00,1.00,0.00,0.00,1.00,0.00,-23.55,-46.64
75%,360000.00,835.00,94.00,3.00,2.00,1.00,2.00,1.00,0.00,1.00,0.00,-23.52,-46.56
max,10000000.00,9500.00,880.00,10.00,8.00,6.00,9.00,1.00,1.00,1.00,1.00,0.00,0.00


In [6]:
pd.Series([i[1].split('/')[1] for i in df.District.iteritems()]).value_counts()

São Paulo    13640
dtype: int64

In [7]:
pd.Series([i[1].split('/')[0] for i in df.District.iteritems()]).value_counts()

Moema              293
Mooca              288
Brás               255
Brooklin           250
Bela Vista         250
                  ... 
Jardim São Luis     44
São Rafael          44
Grajaú              37
São Domingos        35
Perus               30
Length: 96, dtype: int64

In [0]:
df['District'] = df['District'].apply(lambda x: x.split('/')[0])

In [9]:
df['Negotiation Type'].value_counts()

rent    7228
sale    6412
Name: Negotiation Type, dtype: int64

In [10]:
df['Property Type'].value_counts()

apartment    13640
Name: Property Type, dtype: int64

In [0]:
df.drop('Property Type', axis = 1, inplace = True)

#Criando novas Variáveis:

In [13]:
for setor in df.District.unique():
  df[setor] = 0
  df[df.District == setor][setor] = 1
df.drop('District', axis = 1, inplace = True)
df.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(13640, 110)

#Criação do Data Frame `Rent` e `Sale`:

In [14]:
Rent = df[df['Negotiation Type'] == 'rent'].drop('Negotiation Type', axis = 1)
Rent.head(1)

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,Latitude,Longitude,Artur Alvim,Belém,Cangaíba,Carrão,Cidade Líder,Cidade Tiradentes,Ermelino Matarazzo,Iguatemi,Itaim Paulista,Itaquera,Jardim Helena,José Bonifácio,Lajeado,Parque do Carmo,Penha,Ponte Rasa,Sapopemba,São Lucas,São Mateus,São Miguel,São Rafael,Tatuapé,Vila Curuçá,Vila Formosa,Vila Jacuí,Vila Matilde,Vila Prudente,...,Freguesia do Ó,Jaguaré,Jaraguá,Lapa,Limão,Perdizes,Pinheiros,Pirituba,Raposo Tavares,Rio Pequeno,Vila Leopoldina,Barra Funda,Bela Vista,Bom Retiro,Brás,Consolação,Liberdade,Pari,República,Santa Cecília,Sé,Casa Verde,Jaçanã,Mandaqui,Santana,Tremembé,Tucuruvi,Vila Guilherme,Vila Maria,Medeiros,Água Rasa,Aricanduva,Guaianazes,Jardim São Luis,Vila Madalena,Mooca,Brooklin,Vila Olimpia,Perus,São Domingos
0,930,220,47,2,2,1,1,0,0,0,0,-23.543138,-46.479486,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
Sale = df[df['Negotiation Type'] == 'sale'].drop('Negotiation Type', axis = 1)
Sale.head(1)

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,Latitude,Longitude,Artur Alvim,Belém,Cangaíba,Carrão,Cidade Líder,Cidade Tiradentes,Ermelino Matarazzo,Iguatemi,Itaim Paulista,Itaquera,Jardim Helena,José Bonifácio,Lajeado,Parque do Carmo,Penha,Ponte Rasa,Sapopemba,São Lucas,São Mateus,São Miguel,São Rafael,Tatuapé,Vila Curuçá,Vila Formosa,Vila Jacuí,Vila Matilde,Vila Prudente,...,Freguesia do Ó,Jaguaré,Jaraguá,Lapa,Limão,Perdizes,Pinheiros,Pirituba,Raposo Tavares,Rio Pequeno,Vila Leopoldina,Barra Funda,Bela Vista,Bom Retiro,Brás,Consolação,Liberdade,Pari,República,Santa Cecília,Sé,Casa Verde,Jaçanã,Mandaqui,Santana,Tremembé,Tucuruvi,Vila Guilherme,Vila Maria,Medeiros,Água Rasa,Aricanduva,Guaianazes,Jardim São Luis,Vila Madalena,Mooca,Brooklin,Vila Olimpia,Perus,São Domingos
4901,732600,1000,74,1,2,1,2,1,0,1,0,-23.552129,-46.692244,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#Retirando Outliers:

In [16]:
def tirar_outlier(df, lista):
  df_clean = df[lista]
  for col, dados in df_clean.iteritems():
    df_clean = df_clean[((df_clean[col] - df_clean[col].mean()) / df_clean[col].std()).abs() < 3]
    df_clean = df_clean[np.abs(df_clean[col]-df_clean[col].mean()) <= (3*df_clean[col].std())]

  valores_retirados = df.shape[0] - df_clean.shape[0]
  print('Foi retirado {} valores de {} ({:.2f})%'.format(valores_retirados, df.shape[0], valores_retirados/df.shape[0]*100))
  print('Ficamos com {} resultados'.format(df_clean.shape[0]))

  for i in range(0, len(df)):
    if i not in df_clean.index:
        df = df[df.index != i]

  df = df.reset_index(drop=True)
  return df

lista = ['Latitude', 'Longitude']

print('Para o Data Frame Rent:')
Rent_clean = tirar_outlier(Rent, lista)

print('\nPara o Data Frame Sale:')
Sale_clean = tirar_outlier(Sale, lista)

Para o Data Frame Rent:
Foi retirado 577 valores de 7228 (7.98)%
Ficamos com 6651 resultados

Para o Data Frame Sale:
Foi retirado 472 valores de 6412 (7.36)%
Ficamos com 5940 resultados


#Modelos de Machine Learning:

##Primeiro Modelo: Regressão Linear

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

###Para o Data Frame `Rent`:

In [34]:
X = Rent_clean.drop(['Price'], axis = 1)
y = Rent_clean[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
modelo = LinearRegression()
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

print('R² = {:.3f}'.format(r2_score(y_test, y_pred)))

R² = 0.572


###Para o Data Frame `Sale`:

In [23]:
X = Sale_clean.drop(['Price'], axis = 1)
y = Sale_clean[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
modelo = LinearRegression()
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

print('R² = {:.3f}'.format(r2_score(y_test, y_pred)))

R² = 0.725


##Segundo Modelo: XGBRegressor

In [0]:
from xgboost import XGBRegressor

###Para o Data Frame `Rent`:

In [25]:
X = Rent_clean.drop(['Price'], axis = 1)
y = Rent_clean[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
modelo = XGBRegressor(random_state=42, silent=True)
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

print('R² = {:.3f}'.format(r2_score(y_test, y_pred)))

R² = 0.725


###Para o Data Frame `Sale`:

In [26]:
X = Sale_clean.drop(['Price'], axis = 1)
y = Sale_clean[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
modelo = XGBRegressor(random_state=42, silent=True)
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

print('R² = {:.3f}'.format(r2_score(y_test, y_pred)))

R² = 0.856


##Terceiro Modelo: Essemble Random Forest Regressor

In [0]:
from sklearn.ensemble import RandomForestRegressor

###Para o Data Frame `Rent`:

####Sem alterações:

In [32]:
X = Rent_clean.drop(['Price'], axis = 1)
y = Rent_clean[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

modelo = RandomForestRegressor()
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

print('R² = {:.3f}'.format(r2_score(y_test, y_pred)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


R² = 0.778


####Utilizando "`mae`" (Mean Absolute Error):

In [33]:
X = Rent_clean.drop(['Price'], axis = 1)
y = Rent_clean[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

modelo = RandomForestRegressor(10, criterion = 'mae')
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

print('R² = {:.3f}'.format(r2_score(y_test, y_pred)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


R² = 0.722


In [36]:
X = Rent_clean.drop(['Price'], axis = 1)
y = Rent_clean[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

modelo = RandomForestRegressor(min_samples_split=5, min_samples_leaf=5)
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

print('R² = {:.3f}'.format(r2_score(y_test, y_pred)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


R² = 0.742
